In [1]:
%pip install qiskit

Note: you may need to restart the kernel to use updated packages.


In [1]:
from qiskit import *
from qiskit.visualization import *
from qiskit.circuit.library import MCPhaseGate
import qiskit.quantum_info as qi
import numpy as np
import pandas
import matplotlib.pyplot as plt


from random import *
from math import *

sim = Aer.get_backend('aer_simulator')
sim.set_options(device = 'GPU')

In [2]:

def phase_normalize(alfa,a):
  return (alfa / a)  * (pi/2)

def phase_denormalize(norm,a):
  return ((norm * a) / (pi/2))

def Uw_phi(basis,phi,n):
  reg_q = QuantumRegister(n,"qr")
  circ_Uwp = QuantumCircuit(reg_q)
  neg = []
  for i,bit in enumerate(basis):
    if bit == "0":
      neg.append(i)
      circ_Uwp.x(i)
  circ_Uwp.append(MCPhaseGate(-phi,n-1),reg_q)
  for i in neg:
    circ_Uwp.x(i)
  circ_Uwp.barrier()
  return circ_Uwp

def Uw(W_phi, n):
  reg_q = QuantumRegister(n,"qr")
  circ_Uw = QuantumCircuit(reg_q)

  for i in range(1,len(W_phi)):
    basis = bin(i)[2:]
    while(len(basis) < n):
      basis = '0' + basis
    circ_Uw.compose(Uw_phi(basis,W_phi[i] - W_phi[0],n),reg_q,inplace = True)

  circ_Uw.h(reg_q)
  return circ_Uw

def Ui_theta(basis,theta,n):
  reg_q = QuantumRegister(n,"qr")
  circ_Uit = QuantumCircuit(reg_q)
  neg = []
  for i,bit in enumerate(basis):
    if bit == "0":
      neg.append(i)
      circ_Uit.x(i)
  circ_Uit.append(MCPhaseGate(theta,n-1),reg_q)
  for i in neg:
    circ_Uit.x(i)
  circ_Uit.barrier()
  return circ_Uit

def Ui(train_set_x, n):
  reg_q = QuantumRegister(n,"qr")
  circ_Ui = QuantumCircuit(reg_q)
  circ_Ui.h(reg_q)
  for i in range(1,len(train_set_x)):
    basis = bin(i)[2:]
    while(len(basis) < n):
      basis = '0' + basis
    circ_Ui.compose(Ui_theta(basis,train_set_x[i] - train_set_x[0],n),reg_q,inplace = True)
  return circ_Ui


def PerceptronCircuit(train_set_x, w_set, n):
    reg_q = QuantumRegister(n,"qr")
    reg_aux = QuantumRegister(1, "aux")
    cla_aux = ClassicalRegister(1)
    total = n + 1
    circ_perc = QuantumCircuit(reg_q,reg_aux,cla_aux)

    circ_perc.barrier()
    circ_perc.compose(Ui(train_set_x,n),range(n), inplace = True)
    circ_perc.compose(Uw(w_set,n),range(n), inplace = True)
    circ_perc.x(reg_q)
    circ_perc.mcx(reg_q,reg_aux)
    circ_perc.measure(reg_aux,cla_aux)
    return circ_perc




In [5]:

def initialize(train_set_x,sz):
    w = np.array([uniform(0,pi/2) for i in range(sz)])
    train_set_x = phase_normalize(train_set_x,255)
    return w, train_set_x

# tem que estar normalizado
def get_activation(w,train_set_x,num_shots = 8192):
    sz = len(train_set_x[0])
    n = ceil(log2(sz))
    A = np.array([])
    for pos,train_input in enumerate(train_set_x):
        model = PerceptronCircuit(train_input, w, n)
        qobj = assemble(model)   
        counts = sim.run(qobj,shots = num_shots).result().get_counts()
        if "1" in counts:
            A = np.append(A,counts["1"]/ num_shots)
        else:
            A = np.append(A,0.)
    return A

def get_dJ(w,train_set_x,n):
    u = train_set_x - w     # theta - phi
    c = np.cos(u)           # cos(theta - phi)
    C = np.sum(c)
    s = np.sin(u)
    S = np.sum(s)
    return 2 * (s * C - c * S)/2**(2 * n)

def get_A(w,train_set_x,n):
    u = train_set_x - w     # theta - phi
    c = np.cos(u)           # cos(theta - phi)
    C = np.sum(c)
    s = np.sin(u)
    S = np.sum(s)
    return (C * C + S * S)/2**(2 * n)

def propagate(w,train_set_x,train_set_y):
    A = get_activation(w,train_set_x)
    diff = A - train_set_y
    cost = np.sum(diff ** 2)
    dA = get_dA(w,train_set_x,n)
    dw = 2 * dA * (diff)
    return dw, cost

def optimize(w,train_set_x,train_set_y, Hyperparameters):
    for i in range(Hyperparameters["iterations"]):
       dw, cost = propagate(w,train_set_x,train_set_y, Hyperparameters["num_shots"])
       w = w - (Hyperparameters["learning_rate"] * dw)
            
    return w

In [4]:
import fiftyone as fo
import fiftyone.zoo as foz


dataset_train = foz.load_zoo_dataset(
    "open-images-v6",
    split = "train",
    label_types = ["classifications"],
    classes = ["Cat"],
    max_samples = 500,

)
dataset_test = foz.load_zoo_dataset(
    "open-images-v6",
    split = "test",
    label_types = ["classifications"],
    classes = ["Cat"],
    max_samples = 500,
)


NumExpr defaulting to 8 threads.
Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading 'open-images-v6' split 'train'
 100% |█████████████████| 500/500 [2.8s elapsed, 0s remaining, 248.5 samples/s]      
Dataset 'open-images-v6-train-500' created
Necessary images already downloaded
Existing download of split 'test' is sufficient
Loading 'open-images-v6' split 'test'
 100% |█████████████████| 500/500 [1.4s elapsed, 0s remaining, 346.5 samples/s]         
Dataset 'open-images-v6-test-500' created


In [19]:

from PIL import Image,ImageOps

def load_params():
    w = np.load("./params/w.npy")
    Hyperparameters = np.load("./params/Hyperparameters.npy")
    return w, Hyperparameters
    

def load_images(path):
    images = np.array([])
    images = images.reshape(0,16 * 16)
    for filename in os.listdir(path):
        f = os.path.join(path, filename)
        if os.path.isfile(f):
            image_np = load_image(f)
            images = np.vstack((images,image_np[0]))
    return images

def load_image(f):
    image = ImageOps.grayscale(Image.open(f))
    image = image.resize((16,16))
    image_np = np.array(image)
    image_np = image_np.reshape(1,16 * 16)
    return image_np


def prepare_images_labels(dataset):
    #set_y = np.zeros(len(dataset))
    set_y = np.zeros(50)
    set_x = np.array([])
    set_x = set_x.reshape(0,16 * 16)
    for pos,i in enumerate(dataset):
        image_np = load_image(i["filepath"])
        set_x = np.vstack((set_x,image_np[0]))
        for j in range(len(i.positive_labels.to_dict()["classifications"])):
            if i.positive_labels.to_dict()["classifications"][j]['label'] == "Cat":
                set_y[pos] = 1
                break
        if pos + 1 == 50:
            break
    return set_x, set_y

def load_data(dataset_train,dataset_test):
    train_set_x, train_set_y = prepare_images_labels(dataset_train)
    test_set_x, test_set_y = prepare_images_labels(dataset_test)
    return train_set_x, train_set_y, test_set_x, test_set_y
    
train_set_x, train_set_y, test_set_x, test_set_y = load_data(dataset_train,dataset_test)    

In [38]:
from qiskit.algorithms.optimizers import SPSA

m = len(train_set_x[0])
set_size = len(train_set_y)
n = ceil(log2(sz))
w, train_set_x = initialize(train_set_x,m)
Hyperparameters = {
"iterations" : 100,
"threshold" : 0.5,
}
print(set_size)
def powerseries(eta=0.01, power=2, offset=0):
    n = 1
    while True:
        yield eta / ((n + offset) ** power)
        n += 1

def learning_rate():
    return powerseries(0.06188218461851615, 0.602, 0)

def perturbation():
    return powerseries(0.2,  0.101)

spsa = SPSA(maxiter=Hyperparameters["iterations"]) #,learning_rate=learning_rate,perturbation=perturbation)
def get_dJ(w):
    u = train_set_x - w     # theta - phi
    c = np.cos(u)           # cos(theta - phi)
    C = c.sum(axis = 1).reshape(1,m).T
    s = np.sin(u)
    S = s.sum(axis = 1).reshape(1,m).T
    A = (C * C + S * S) / 2**(2 * n)
    return (2 * (s*C - c*S) / 2**(2 * n)) * A

def loss_func(w):
    A = get_activation(w,train_set_x)
    diff = A - train_set_y
    cost = np.sum(diff ** 2)
    return cost

m = len(train_set_x[0])

result = spsa.optimize(num_vars=1,objective_function=loss_func,gradient_function=get_dJ, initial_point = w)

50
SPSA: Starting calibration of learning rate and perturbation.
Total Assembly Time - 0.18859 (ms)
Iter: 0 
Total Assembly Time - 0.13041 (ms)
Iter: 1 
Total Assembly Time - 0.10347 (ms)
Iter: 2 
Total Assembly Time - 0.10395 (ms)
Iter: 3 
Total Assembly Time - 0.07391 (ms)
Iter: 4 
Total Assembly Time - 0.18191 (ms)
Iter: 5 
Total Assembly Time - 0.07796 (ms)
Iter: 6 
Total Assembly Time - 0.09036 (ms)
Iter: 7 
Total Assembly Time - 0.13328 (ms)
Iter: 8 
Total Assembly Time - 0.17262 (ms)
Iter: 9 
Total Assembly Time - 0.71502 (ms)
Iter: 10 
Total Assembly Time - 0.17619 (ms)
Iter: 11 
Total Assembly Time - 0.17715 (ms)
Iter: 12 
Total Assembly Time - 0.08845 (ms)
Iter: 13 
Total Assembly Time - 0.12565 (ms)
Iter: 14 
Total Assembly Time - 0.07415 (ms)
Iter: 15 
Total Assembly Time - 0.10133 (ms)
Iter: 16 
Total Assembly Time - 0.07296 (ms)
Iter: 17 
Total Assembly Time - 0.08798 (ms)
Iter: 18 
Total Assembly Time - 0.15235 (ms)
Iter: 19 
Total Assembly Time - 0.07463 (ms)
Iter: 20 


KeyboardInterrupt: 

In [36]:
print(spsa.learning_rate)
print(result)

<function learning_rate at 0x7f84394ecb80>
(array([0.0627059 , 1.4636975 , 1.5551055 , 1.51896594, 0.3019329 ,
       0.77098868, 0.31221468, 0.38891644, 0.79167969, 0.43493657,
       0.48909778, 1.01360522, 0.14649317, 1.10897414, 1.50038428,
       1.10936985, 0.29485277, 1.03447614, 0.03123437, 0.78155582,
       0.57514429, 1.30794421, 0.77729846, 0.73424001, 0.81096222,
       0.77699192, 0.8022223 , 1.05726053, 0.2239568 , 0.42922708,
       1.08157375, 0.0496719 , 0.55679024, 0.58367625, 1.45327287,
       0.03733408, 0.38002274, 0.45490741, 0.03432896, 0.54758442,
       1.5388279 , 1.31986899, 1.26703583, 1.13029185, 0.18030551,
       0.50739826, 1.01343523, 1.47258424, 0.62498797, 1.22631329,
       0.61436321, 0.50339301, 1.54370514, 1.4093835 , 0.65863409,
       0.48718205, 0.05741838, 0.00742724, 1.25768708, 0.34609044,
       1.20201976, 0.03413243, 0.36117669, 0.58217014, 1.16650265,
       1.4074996 , 0.49444527, 0.36686876, 0.48296694, 0.20465312,
       0.03693763,